# Solving a Multi-Constraint Knapsack Problem with Boundhound

In this tutorial, we'll solve a complex knapsack problem using the boundhound library. This problem demonstrates the power of mixed-integer linear programming (MILP) for solving real-world optimization problems.

## Problem Description

We have 6 items, each with different characteristics:

| Item | Value | Weight | Volume | Cost | Description |
|------|--------|---------|---------|-------|-------------|
| 1    | 30     | 8       | 3       | 7     | High value, high weight, low volume |
| 2    | 28     | 7       | 4       | 6     | Similar to item 1 |
| 3    | 20     | 4       | 7       | 3     | Medium value, low weight, high volume |
| 4    | 18     | 3       | 8       | 2     | Similar to item 3 |
| 5    | 15     | 5       | 5       | 4     | Balanced item |
| 6    | 14     | 6       | 4       | 5     | Balanced item |

### Constraints
1. Total weight must be ≤ 15 (tight - can only take 2-3 items)
2. Total volume must be ≤ 16 (tight - can only take 2-3 items)
3. Total cost must be ≤ 12 (very tight - can only take 2 expensive or 3-4 cheap items)

### Objective
Maximize the total value of selected items.

### What Makes This Problem Interesting?
1. Items come in complementary pairs with different trade-offs:
   - Items 1&2: High value but expensive and heavy
   - Items 3&4: Medium value, light but high volume
   - Items 5&6: Balanced but not optimal for any single metric
2. The LP relaxation will be fractional because:
   - No single item dominates on all metrics
   - Tight constraints force fractional combinations
3. Multiple feasible integer solutions exist with similar values
4. Branch and bound will need to prune branches due to:
   - Weight/volume constraints eliminating heavy/bulky combinations
   - Cost constraint preventing selection of too many expensive items

In [ ]:
import numpy as np
from boundhound.branch_and_bound import solve_milp
from boundhound.core.problem import LPProblem, MILPProblem
from boundhound.types import Sense

# Enable logging to see the solver's progress
import logging
logging.basicConfig(level=logging.INFO)

## Building the Problem

Let's construct our MILP. We'll define:
1. Item characteristics (values, weights, volumes, costs)
2. Capacity constraints
3. Binary decision variables (0 = don't take item, 1 = take item)

The mathematical formulation is:

```
maximize    30x₁ + 28x₂ + 20x₃ + 18x₄ + 15x₅ + 14x₆
subject to   8x₁ +  7x₂ +  4x₃ +  3x₄ +  5x₅ +  6x₆ ≤ 15  (weight)
             3x₁ +  4x₂ +  7x₃ +  8x₄ +  5x₅ +  4x₆ ≤ 16  (volume)
             7x₁ +  6x₂ +  3x₃ +  2x₄ +  4x₅ +  5x₆ ≤ 12  (cost)
             xᵢ ∈ {0,1} for i = 1,...,6
```

In [ ]:
def build_problem():
    """Build the knapsack problem."""
    # Problem parameters
    values = np.array([30, 28, 20, 18, 15, 14])   # Value of each item
    weights = np.array([8, 7, 4, 3, 5, 6])        # Weight of each item
    volumes = np.array([3, 4, 7, 8, 5, 4])        # Volume of each item
    costs = np.array([7, 6, 3, 2, 4, 5])          # Cost of each item
    weight_capacity = 15  # Maximum total weight
    volume_capacity = 16  # Maximum total volume
    cost_capacity = 12    # Maximum total cost

    # Variables: [x₁, x₂, x₃, x₄, x₅, x₆]
    n_vars = 6

    # Objective: maximize total value
    c = values

    # Constraints: weight, volume, and cost limits
    A_ub = np.vstack([weights, volumes, costs])
    b_ub = np.array([weight_capacity, volume_capacity, cost_capacity])

    # Variable bounds (binary: 0 ≤ xᵢ ≤ 1)
    lb = np.zeros(n_vars)
    ub = np.ones(n_vars)

    # Create the LP and MILP problems
    lp = LPProblem(
        c=c,
        A_ub=A_ub,
        b_ub=b_ub,
        lb=lb,
        ub=ub,
        sense=Sense.MAX,
    )
    return (
        MILPProblem(lp=lp, integer_vars=set(range(n_vars))),
        values,
        weights,
        volumes,
        costs,
    )

# Build the problem
milp, values, weights, volumes, costs = build_problem()

## Solving the Problem

Now let's solve the problem using boundhound's branch and bound solver. We'll set a maximum of 100 nodes to explore, though we expect to find the optimal solution much earlier.

The solver will:
1. Start with the LP relaxation at the root node
2. Branch on fractional variables
3. Prune branches that are infeasible or worse than the best known solution
4. Continue until an optimal integer solution is found

In [ ]:
def print_solution(solution, values, weights, volumes, costs):
    """Print the solution in a readable format."""
    if solution.x is None:
        print("No solution found")
        return

    print("\nSolution Details:")
    print("-" * 30)
    print(f"Objective value: {solution.value:.2f}")
    print(f"Status: {solution.status.name}")
    print(f"Nodes processed: {solution.nodes_processed}")
    print(f"Nodes remaining: {solution.nodes_remaining}")
    print(f"MIP gap: {solution.mip_gap:.2%}" if solution.mip_gap is not None else "MIP gap: N/A")

    print("\nItem Selections:")
    total_weight = 0
    total_volume = 0
    total_cost = 0
    for i in range(len(solution.x)):
        status = "SELECTED" if solution.x[i] > 0.5 else "NOT SELECTED"
        print(f"Item {i+1}: {status}")
        print(f"  Value: {values[i]:.1f}")
        print(f"  Weight: {weights[i]:.1f}")
        print(f"  Volume: {volumes[i]:.1f}")
        print(f"  Cost: {costs[i]:.1f}")
        if solution.x[i] > 0.5:
            total_weight += weights[i]
            total_volume += volumes[i]
            total_cost += costs[i]

    print(f"\nConstraint Usage:")
    print(f"Total weight: {total_weight:.1f} / {15:.1f} ({total_weight/15*100:.1f}%)")
    print(f"Total volume: {total_volume:.1f} / {16:.1f} ({total_volume/16*100:.1f}%)")
    print(f"Total cost: {total_cost:.1f} / {12:.1f} ({total_cost/12*100:.1f}%)")

# Solve the problem
solution = solve_milp(milp, max_nodes=100)

# Print results
print_solution(solution, values, weights, volumes, costs)

# Visualize the branch and bound tree
solution.render()

## Analysis of Results

Let's analyze what makes this solution optimal:

1. **Selected Items**: The solver has chosen a combination that:
   - Maximizes value while respecting all constraints
   - Balances the trade-offs between high-value/expensive items and lower-value/cheaper items

2. **Constraint Usage**: Notice how:
   - At least one constraint is nearly at capacity (binding constraint)
   - Other constraints have some slack
   - This confirms we've found a corner point of the feasible region

3. **Branch and Bound Performance**:
   - The number of nodes processed shows how efficiently the solver found the solution
   - The MIP gap confirms we've found the global optimum
   - The visualization shows the branching decisions that led to this solution

This example demonstrates how boundhound can solve complex, multi-constraint optimization problems with integer variables, finding provably optimal solutions through intelligent branching and bounding strategies.